<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/notebooks/2020SightVisit_kmnist_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 資格スクエア G 検定対策ビデオ教材 の kmninst プロトタイプ

- filename: `2020SightVisit_kmnist_prototype.ipynb`
- author: 浅川伸一

---



In [ ]:
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz

In [ ]:
import numpy as np

def load(f):
    return np.load(f)['arr_0']

# Load the data
Xkm_train = load('kmnist-train-imgs.npz')
Xkm_test = load('kmnist-test-imgs.npz')
ykm_train = load('kmnist-train-labels.npz')
ykm_test = load('kmnist-test-labels.npz')

# Flatten images
n_samples = 2000
#x_train = x_train.reshape(-1, 784)[:n_samples]
#y_train = y_train[:n_samples]
#x_test = x_test.reshape(-1, 784)

ind2c =[c for c in 'おきすつなまはやれを']

In [ ]:
!pip install japanize-matplotlib
import japanize_matplotlib

In [ ]:
# ライブラリの輸入
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


In [ ]:
# PyTorch image augmentation module
class PyTorchImageDataset(Dataset):
    def __init__(self, image_list, transforms=None):
        self.image_list = image_list
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.image_list))
    
    def __getitem__(self, i):
        image = self.image_list[i]
        image = Image.fromarray(image).convert('RGB')        
        image = np.asarray(image).astype(np.uint8)
        if self.transforms is not None:
            image = self.transforms(image)
            
        return torch.tensor(image, dtype=torch.float)

In [ ]:
pytorch_dataset = PyTorchImageDataset(image_list=Xkm_train, transforms=None)
pytorch_dataloader = DataLoader(dataset=pytorch_dataset, batch_size=16, shuffle=True)

In [ ]:
def show_img(img):
    #plt.figure(figsize=(18,15))
    # unnormalize
    #img = img / 2 + 0.5  
    npimg = img.numpy()
    npimg = np.clip(npimg, 0., 1.)
    plt.imshow(np.transpose(npimg, (0, 1, 2)))
    plt.show()

In [ ]:
data = iter(pytorch_dataloader)
images = data.next()

# show images
#plt.imshow(np.asarray(images[0].numpy().astype(np.uint8)))
show_img(images[0])

In [ ]:
images[0].shape

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)  # 引数の意味は 入力チャンネル（特徴）数，出力チャンネル数, カーネルサイズ，ストライド の 4 つを指定します
        self.conv2 = nn.Conv2d(32, 64, 3, 1) # 従って直上の nn.Conv2d() の第2引数と，この行の第一引数が等しい必要があります。
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128) # 9216 の計算だけ面倒です。
        # 9216 の心は 入力画像が 28 x 28 で 3 x 3 の畳み込み(ストライド1)を 2 回かけるので，画像は 28x28 -> 26x26 -> 24x24 に小さくなっています。
        # そこで 64 チャンネルあるので 24 x 24 x 64 = 36864
        # これに 2 x 2 のマックスプーリングをかけるので 1/4 に減ります。
        # すなわち 24 x 24 x 64 / 4 = 9216 です
        self.fc2 = nn.Linear(128, 10) # 下位層からの入力次元が 128 で出力次元が 10

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


In [ ]:
net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(pytorch_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        #inputs, labels = data
        inputs = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
import time

start = time.time()
for i, data in enumerate(pytorch_dataloader):
    images = data
    outputs = net(images)
end = time.time()
time_spent = (end-start)/60
print(f"{time_spent:.3} minutes")

In [ ]:
import math

weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)


In [ ]:
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)

def model(xb):
    return log_softmax(xb @ weights + bias)

In [ ]:
bs = 64  # batch size

xb = Xkm_train[0:bs]  # a mini-batch from x
preds = model(xb)  # predictions
preds[0], preds.shape
print(preds[0], preds.shape)